# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 17 2022 9:13AM,248707,12,HH-37530,Hush Hush,Released
1,Oct 17 2022 9:13AM,248707,12,HH-37531,Hush Hush,Released
2,Oct 17 2022 9:13AM,248707,12,HH-37532,Hush Hush,Released
3,Oct 17 2022 9:13AM,248707,12,HH-37533,Hush Hush,Released
4,Oct 17 2022 9:13AM,248707,12,HH-37534,Hush Hush,Released
5,Oct 17 2022 9:13AM,248707,12,HH-37535,Hush Hush,Released
6,Oct 17 2022 9:13AM,248707,12,HH-37536,Hush Hush,Released
7,Oct 17 2022 9:13AM,248707,12,HH-37537,Hush Hush,Released
8,Oct 17 2022 9:13AM,248707,12,HH-37538,Hush Hush,Released
9,Oct 17 2022 9:13AM,248707,12,HH-37539,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
10,248705,Released,1
11,248706,Released,13
12,248707,Released,55
13,248708,Released,1
14,248709,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
248705,NaN,1.0
248706,NaN,13.0
248707,NaN,55.0
248708,NaN,1.0
248709,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
248690,6.0,1.0
248691,0.0,23.0
248697,0.0,1.0
248698,0.0,6.0
248699,0.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
248690,6,1
248691,0,23
248697,0,1
248698,0,6
248699,0,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,248690,6,1
1,248691,0,23
2,248697,0,1
3,248698,0,6
4,248699,0,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,248690,6,1
1,248691,,23
2,248697,,1
3,248698,,6
4,248699,,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 17 2022 9:13AM,248707,12,Hush Hush
55,Oct 17 2022 9:12AM,248709,19,"Granules Pharmaceuticals, Inc."
56,Oct 17 2022 9:07AM,248708,10,Bio-PRF
57,Oct 17 2022 9:05AM,248706,10,ISDIN Corporation
70,Oct 17 2022 9:00AM,248705,10,"Virtue Rx, LLC"
71,Oct 17 2022 8:59AM,248704,10,Emerginnova
72,Oct 17 2022 8:50AM,248702,10,Eye Pharma Inc
78,Oct 17 2022 8:30AM,248690,10,ISDIN Corporation
85,Oct 17 2022 8:30AM,248691,10,ISDIN Corporation
108,Oct 17 2022 8:26AM,248700,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 17 2022 9:13AM,248707,12,Hush Hush,,55
1,Oct 17 2022 9:12AM,248709,19,"Granules Pharmaceuticals, Inc.",,1
2,Oct 17 2022 9:07AM,248708,10,Bio-PRF,,1
3,Oct 17 2022 9:05AM,248706,10,ISDIN Corporation,,13
4,Oct 17 2022 9:00AM,248705,10,"Virtue Rx, LLC",,1
5,Oct 17 2022 8:59AM,248704,10,Emerginnova,,1
6,Oct 17 2022 8:50AM,248702,10,Eye Pharma Inc,,6
7,Oct 17 2022 8:30AM,248690,10,ISDIN Corporation,6,1
8,Oct 17 2022 8:30AM,248691,10,ISDIN Corporation,,23
9,Oct 17 2022 8:26AM,248700,16,Sartorius Bioprocess Solutions,2,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 9:13AM,248707,12,Hush Hush,55,
1,Oct 17 2022 9:12AM,248709,19,"Granules Pharmaceuticals, Inc.",1,
2,Oct 17 2022 9:07AM,248708,10,Bio-PRF,1,
3,Oct 17 2022 9:05AM,248706,10,ISDIN Corporation,13,
4,Oct 17 2022 9:00AM,248705,10,"Virtue Rx, LLC",1,
5,Oct 17 2022 8:59AM,248704,10,Emerginnova,1,
6,Oct 17 2022 8:50AM,248702,10,Eye Pharma Inc,6,
7,Oct 17 2022 8:30AM,248690,10,ISDIN Corporation,1,6
8,Oct 17 2022 8:30AM,248691,10,ISDIN Corporation,23,
9,Oct 17 2022 8:26AM,248700,16,Sartorius Bioprocess Solutions,1,2


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 9:13AM,248707,12,Hush Hush,55,
1,Oct 17 2022 9:12AM,248709,19,"Granules Pharmaceuticals, Inc.",1,
2,Oct 17 2022 9:07AM,248708,10,Bio-PRF,1,
3,Oct 17 2022 9:05AM,248706,10,ISDIN Corporation,13,
4,Oct 17 2022 9:00AM,248705,10,"Virtue Rx, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 9:13AM,248707,12,Hush Hush,55,NaN
1,Oct 17 2022 9:12AM,248709,19,"Granules Pharmaceuticals, Inc.",1,NaN
2,Oct 17 2022 9:07AM,248708,10,Bio-PRF,1,NaN
3,Oct 17 2022 9:05AM,248706,10,ISDIN Corporation,13,NaN
4,Oct 17 2022 9:00AM,248705,10,"Virtue Rx, LLC",1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 9:13AM,248707,12,Hush Hush,55,0.0
1,Oct 17 2022 9:12AM,248709,19,"Granules Pharmaceuticals, Inc.",1,0.0
2,Oct 17 2022 9:07AM,248708,10,Bio-PRF,1,0.0
3,Oct 17 2022 9:05AM,248706,10,ISDIN Corporation,13,0.0
4,Oct 17 2022 9:00AM,248705,10,"Virtue Rx, LLC",1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2238303,59,6.0
12,248707,55,0.0
16,248700,1,2.0
19,248709,1,0.0
20,248697,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2238303,59,6.0
1,12,248707,55,0.0
2,16,248700,1,2.0
3,19,248709,1,0.0
4,20,248697,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,59,6.0
1,12,55,0.0
2,16,1,2.0
3,19,1,0.0
4,20,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,59.0
1,12,Released,55.0
2,16,Released,1.0
3,19,Released,1.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20
Status,,,,,
Executing,6.0,0.0,2.0,0.0,0.0
Released,59.0,55.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20
0,Executing,6.0,0.0,2.0,0.0,0.0
1,Released,59.0,55.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20
0,Executing,6.0,0.0,2.0,0.0,0.0
1,Released,59.0,55.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()